In [4]:
import cv2 
import os

In [23]:
# Function to extract frames

def extract_frames(vid_path):

    videoName = vid_path.split("/")[-1].split(".mp4")[0]
    directory = "extracted_frames/" + videoName

    if not os.path.exists(directory):
        os.mkdir(directory)
    
    # Path to video file
    videoObj = cv2.VideoCapture(vid_path)
  
    # Used as counter variable
    count = 1
  
    # checks whether frames were extracted
    success = 1
  
    while success:
        success, image = videoObj.read()
        if image is not None:
        # Saves the frames with frame-count
            cv2.imwrite(directory + "/" + str(count) + ".jpg", image)
  
        count += 1
    print(count)

In [24]:
directory = r"C:/Harrithha/IIT_Dh/College_Academics/Sem_6/MDS/MDS_2023/Assignment_1/videos/"

for video in os.listdir(directory):
    videoPath  = directory + video
    extract_frames(videoPath)

458


In [29]:
import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt
  
# assign and open image
url = 'C:/Harrithha/IIT_Dh/College_Academics/Sem_6/MDS/MDS_2023/Assignment_1/images/1.png'

img = cv2.imread(url)

# Converting the image into gray scale for faster
# computation.
  
# Calculating the SVD
u, s, v = np.linalg.svd(img, full_matrices=False)
  
# inspect shapes of the matrices
print(f'u.shape:{u.shape},s.shape:{s.shape},v.shape:{v.shape}')

u.shape:(100, 100, 3),s.shape:(100, 3),v.shape:(100, 3, 3)


In [40]:
def findK(percentClarity, arr):
    k = 0
    s = 0
    arrSum = sum(arr)
    for i in arr:
        s += i
        p = (s/arrSum)*100
        k+=1
        if round(p) >= percentClarity:
            return k